## TODO

- [ ] make as many subplots as there are data per day
- [ ] interactive plot where user can choose:
    - [ ] the day to plot: "Monday", "Tuesday", ..., "All"
    - [ ] how to aggregate data: "no aggregation", "mean (+ std ?)", "mean + quantiles (quartiles or deciles)"

In [ ]:
%matplotlib notebook
import os
import pandas as pd

In [ ]:
def data_files_in_dir(directory_path):
    """Return the path of CSV files in `directory_path`.

    Return the path of all (or `max_num_files`) files having the extension
    ".simtel", ".simtel.gz", ".fits" or ".fit" in `directory_path`.

    Parameters
    ----------
    directory_path : str
        The directory's path where CSV files are searched.

    Yields
    ------
    str
        The path of the next CSV file in `directory_path`.
    """

    FILE_EXT = (".csv", )
    directory_path = os.path.expanduser(directory_path)

    for file_name in os.listdir(directory_path):
        file_path = os.path.join(directory_path, file_name)
        if os.path.isfile(file_path) and file_name.lower().endswith(FILE_EXT):
            yield file_path

In [ ]:
file_path_list = data_files_in_dir("data")


# https://machinelearningmastery.com/load-explore-time-series-data-python/
#    header=0: We must specify the header information at row 0.
#    parse_dates=[0]: We give the function a hint that data in the first column contains dates that need to be parsed. This argument takes a list, so we provide it a list of one element, which is the index of the first column.
#    index_col=0: We hint that the first column contains the index information for the time series.
#    squeeze=True: We hint that we only have one data column and that we are interested in a Series and not a DataFrame.


# "squeeze=True" is used to return a TimeSeries (instead a DataFrame with only one column) ; see http://pandas.pydata.org/pandas-docs/version/0.22/generated/pandas.Series.from_csv.html
ts = pd.concat((pd.read_csv(file_path, index_col=0, parse_dates=[0], squeeze=True) for file_path in file_path_list))
print(ts)

In [ ]:
ax = ts.plot(y='duration', figsize=(14, 7))
ax.set_xlabel('Time')
ax.set_ylabel('Trip duration (mn)');

## TODO: save figure in a png file ?

In [ ]:
ts.index

In [ ]:
ts.resample('1B').mean().plot()

In [ ]:
ts2 = ts.resample('1min', kind='period').mean()

In [ ]:
ts2.index

In [ ]:
# https://stackoverflow.com/questions/16266019/python-pandas-group-datetime-column-into-hour-and-minute-aggregations

#ts2.groupby([ts2.index.hour, ts2.index.minute]).mean()

In [ ]:
# https://jakevdp.github.io/PythonDataScienceHandbook/03.11-working-with-time-series.html#Digging-into-the-data

ax = ts2.groupby(ts2.index.start_time.time).mean().plot(y='duration', figsize=(18, 12), color="red")
ts2.groupby(ts2.index.start_time.time).median().plot(ax=ax, color="green")
ts2.groupby(ts2.index.start_time.time).quantile(0.25).plot(ax=ax, color="blue", alpha=0.75, style="--")
ts2.groupby(ts2.index.start_time.time).quantile(0.75).plot(ax=ax, color="blue", alpha=0.75, style="--")
ts2.groupby(ts2.index.start_time.time).quantile(0.95).plot(ax=ax, color="blue", alpha=0.5, style=":")
ts2.groupby(ts2.index.start_time.time).quantile(0.05).plot(ax=ax, color="blue", alpha=0.5, style=":")

ax.set_xlabel('Time')
ax.set_ylabel('Trip duration (mn)');

In [ ]:
pd.DataFrame(ts2.index.start_time.time)